Rayence Image reconstruction

This demo will show you a simple image reconstruction can be performed, by using OS_SART and FDK.


## Define geometry

## Load data and generate projections

## Reconstruct image using OS-SART and FDK

In [ ]:
import tigre
import numpy as np
import os
from PIL import Image
import json
# from tigre.demos.Test_data import data_loader
import nibabel as nib
# from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift, gaussian_filter
import matplotlib.pyplot as plt
import time
import cv2
from scipy import ndimage, interpolate
import pylab# as plt
geo = tigre.geometry_jasper(high_quality=True)
geosmall = tigre.geometry_jasper(high_quality=False) 
tigre.plot_geometry(geo, 0)
import scipy.optimize
import tigre.algorithms as algs
import tomopy
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans
import math


def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = np.array(tt)
    yy = np.array(yy)
    ff = np.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(np.fft.fft(yy))
    guess_freq = abs(ff[np.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = np.std(yy) * 2.**0.5
    guess_offset = np.mean(yy)
    guess = np.array([guess_amp, 2.*np.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * np.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*np.pi)
    fitfunc = lambda t: A * np.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": np.max(pcov), "rawres": (guess,popt,pcov)}

In [ ]:
# drive = 'c:\\'
# projdirname = os.path.join(drive,'data','mpx3')
# projcorrdirname = projdirname

drive = 'f:\\'

projdirname = os.path.join(drive,'jasper','data','20220720_movingdetector', 'motion_to_5pix', '20220720113833_8_10')
openfielddirname = os.path.join(drive,'jasper','data','20220720_movingdetector','Open_no_motion','20220720130159_8_10')
badpixelcorrdirname = os.path.join(projdirname, 'BPC_images_08')
projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08')#_energy5')
# projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08_energy5')
# projcorrdirname = projdirname
resultimages = os.path.join(projdirname,'results')
print(projcorrdirname)
print(openfielddirname)
if not os.path.exists(resultimages):
    os.makedirs(resultimages)
jsonfile = os.path.join(projdirname,'scan_8_10_keV.json')
    


In [ ]:
drive = 'f:\\'

projdirname = os.path.join(drive,'jasper','data','20220720_ham_n_cheese', 'motion_to_5pix', '8_10')
openfielddirname = os.path.join(drive,'jasper','data','20220720_ham_n_cheese','motion_to_5pix','open_8_10')
badpixelcorrdirname = os.path.join(projdirname, 'BPC_images_08')
projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08')#_energy5')
# projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08_energy5')
# projcorrdirname = projdirname
resultimages = os.path.join(projdirname,'results')
print(projcorrdirname)
print(openfielddirname)
if not os.path.exists(resultimages):
    os.makedirs(resultimages)
jsonfile = os.path.join(projdirname,'scan_8_10_keV.json')
    


In [ ]:
drive = 'f:\\'

projdirname = os.path.join(drive,'jasper','data','20220720_movingdetector', 'motion_to_3pix', '20220720122839_8_10')
openfielddirname = os.path.join(drive,'jasper','data','20220720_movingdetector','Open_no_motion','20220720130159_8_10')
badpixelcorrdirname = os.path.join(projdirname, 'BPC_images_08')
projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08')#_energy5')
# projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08_energy5')
# projcorrdirname = projdirname
resultimages = os.path.join(projdirname,'results')
print(projcorrdirname)
print(openfielddirname)
if not os.path.exists(resultimages):
    os.makedirs(resultimages)
jsonfile = os.path.join(projdirname,'scan_8_10_keV.json')

In [ ]:
drive = 'f:\\'
projdirname = os.path.join(drive,'jasper','data','20220720_movingdetector', 'no_motion', '20220720105125_8_10')
openfielddirname = os.path.join(drive,'jasper','data','20220720_movingdetector','Open_no_motion','20220720130159_8_10')
badpixelcorrdirname = os.path.join(projdirname, 'BPC_images_08')
projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08')#_energy5')
# projcorrdirname = os.path.join(projdirname,'ProjectionsCorr_08_energy5')
# projcorrdirname = projdirname
resultimages = os.path.join(projdirname,'results')
print(projcorrdirname)
print(openfielddirname)
if not os.path.exists(resultimages):
    os.makedirs(resultimages)
jsonfile = os.path.join(drive,'jasper','data','20220720_movingdetector', 'no_motion', '20220720105125_8_10','scan_8_10_keV.json')

In [ ]:
f = open(jsonfile)
dashboard = json.load(f)

# print(dashboard)
detector_x = []
detector_y = []
sample_r = []
for i in dashboard['projections']:
    detector_x.append(dashboard['projections'][i]['detector_x']) 
    detector_y.append(dashboard['projections'][i]['detector_y']) 
    sample_r.append(dashboard['projections'][i]['sample_r']) 
detector_x = np.array(detector_x)
detector_y = np.array(detector_y)
sample_r = np.array(sample_r)
detector_x_offsets = -1*(detector_x-detector_x[0])
detector_y_offsets = -1*(detector_y-detector_y[0])
angles = ((sample_r/360.0)*2*np.pi).astype(np.float32)


In [ ]:

print(angles[249], angles[250])

In [ ]:
if os.path.exists(projcorrdirname):
#     projectionsCorr = data_loader.load_medipix3_files2(projcorrdirname, False, False, True)
    projectionsCorr = tigre.load_medipix(projcorrdirname, flip=False, badpixelcorr=False,  medianfilter=False, halvedata=False, fillgap=False)    
else:
    if os.path.exists(badpixelcorrdirname):
        projections = data_loader.load_medipix3_files2(badpixelcorrdirname, False, False, False)
        openfielddirname = os.path.join(badpixelcorrdirname, 'Openfield')
        openimages = data_loader.load_medipix3_files2(openfielddirname, False, False, False)
    else:
        projections = tigre.load_medipix(projdirname, filterstr='000015_0', flip=False, badpixelcorr=True,  medianfilter=False, halvedata=False, fillgap=False)      
        openimages = tigre.load_medipix(openfielddirname, filterstr='000015_0', flip=False, badpixelcorr=True,  medianfilter=False, halvedata=False, fillgap=False)
        os.makedirs(badpixelcorrdirname)
        for slice in range(0,projections.shape[0]): 
            projection = projections[slice,:,:]
            im = Image.fromarray(projection)
            im.save(os.path.join(badpixelcorrdirname, 'projection{}.tiff'.format(str(slice).zfill(3))))
        openfielddirname = os.path.join(badpixelcorrdirname, 'Openfield')
        os.makedirs(openfielddirname)
        for slice in range(0,openimages.shape[0]): 
            projection = openimages[slice,:,:]
            im = Image.fromarray(projection)
            im.save(os.path.join(openfielddirname, 'openfield{}.tiff'.format(str(slice).zfill(3))))

    flatfield = np.average(openimages, axis=0)
    corr = np.percentile(projections, 99)/np.percentile(flatfield, 99)
    print(corr)
    corr = 1
    flatfield = flatfield*corr
    projectionsCorr = np.empty_like(projections, dtype=np.float64)
    os.makedirs(projcorrdirname)
    for slice in range(0,projections.shape[0]): 
        projection = -np.log(projections[slice,:,:]/flatfield)
        projectionsCorr[slice,:,:] = projection
        im = Image.fromarray(projection)
        im.save(os.path.join(projcorrdirname, 'projection{}.tiff'.format(str(slice).zfill(3))))


In [ ]:
totalproj = projectionsCorr
totalangles = angles
stageoffset = 50
# print(angles)

In [ ]:
# projectionsCorr = tomopy.prep.stripe.remove_stripe_fw(totalproj, level=None, wname='db5', sigma=0.2, pad=True, ncore=None, nchunk=None)
projectionsCorr = tomopy.prep.stripe.remove_all_stripe(totalproj, snr=3, la_size=61, sm_size=21, dim=1, ncore=None, nchunk=None)

In [ ]:
projection0 = projectionsCorr[0,:,:]
projection180 = np.flip(projectionsCorr[round(projectionsCorr.shape[0]/2)-1,:,:],1)
# subpixel precision
shift, error, diffphase = phase_cross_correlation(projection0, projection180,
                                                  upsample_factor=100)

estimate = -shift.item(1)/2
print(estimate)

yshift = 2
projectionsCorrsmall = projectionsCorr[:,int(projectionsCorr.shape[1]/2)-6-yshift:int(projectionsCorr.shape[1]/2)+5-yshift,:]

maxvalue = -9999
opt_shift = -99
N = 140
x = np.zeros(N)
y = np.zeros(N)
cnt =0 
for i in range(int(estimate*100)-70,int(estimate*100)+70,1):
    # note that the detector x and y axis are opposite the x and y coordinate system of the geometry
    geosmall.offDetector = np.vstack((detector_y_offsets, detector_x_offsets+(i/100)*0.055)).T 
    geosmall.offOrigin = np.array((0.0, 0, 0)) 
    geosmall.rotDetector = np.array((math.radians(-0.23),0.0,0.0))
    geosmall.DSD = 192.57
#     stageoffset = 0
    geosmall.DSO = geosmall.DSD-43.74 - stageoffset
    
    imgfdk=algs.fdk(projectionsCorrsmall,geosmall,angles)
    imgfdk[imgfdk < 0] = 0
    imgfdkint = (imgfdk*3000).astype(np.int32)
    im = imgfdkint[0,:,:].astype(np.float32)
    dft = cv2.dft(im, flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
    value = np.mean(magnitude_spectrum)
    if value > maxvalue:
        maxvalue = value
        optshift = i
    x[cnt] = i/100
    y[cnt] = value
    cnt += 1
print(optshift/100, maxvalue)
plt.scatter(x, y)
plt.show()
opt_shift_y =(optshift/100)*0.055

In [ ]:
maxvalue = -9999

opt_shift_x = 0
# note that the detector x and y axis are opposite the x and y coordinate system of the geometry
geo.offDetector = np.vstack((detector_y_offsets+opt_shift_x, detector_x_offsets+opt_shift_y)).T #np.array((opt_shift_x, opt_shift_y)) 

# geo.rotDetector = np.array((math.radians(0.0494658),0.0,0.0))
geo.rotDetector = np.array((math.radians(-0.23),0.0,0.0))

geo.DSD = 192.57
geo.DSO = geo.DSD-43.74 - stageoffset

geo.nVoxel = np.array((512, 512, 512))             # number of voxels              (vx)
geo.sVoxel = np.array((25.6, 25.6, 25.6))          # total size of the image       (mm)
geo.dVoxel = geo.sVoxel/geo.nVoxel                 # size of each voxel            (mm)
geo.offOrigin = np.array((0, 0, 0)) 

start = time.time()
imgfdk=algs.fdk(projectionsCorr,geo,angles)#,filter='cosine')
end = time.time()
print(end - start)

imgfdk[imgfdk < 0] = 0
imgfdkint = (imgfdk*3000).astype(np.int32)
# imgfdkint = tomopy.misc.corr.remove_ring(imgfdkint, center_x=None, center_y=None, thresh=150.0, thresh_max=1500.0, thresh_min=10.0, theta_min=50, rwidth=10, int_mode='WRAP', ncore=None, nchunk=None, out=None)

imgfdkint = np.swapaxes(imgfdkint, 1, 2)
imgfdkint = np.swapaxes(imgfdkint, 0, 2)
imgfdkint = np.flip(imgfdkint, 2)


# header = nib.Nifti1Header()
# header['pixdim'][1:4]  = geo.dVoxel
# ni_img = nib.Nifti1Image(imgfdkint, np.eye(4), header)
ni_img = nib.Nifti1Image(imgfdkint, np.eye(4))
factor = 1.0#/geo.dVoxel[0]
# ni_img.header['pixdim'][1:4]  = geo.dVoxel *factor
nib.save(ni_img,  os.path.join(resultimages,'recon_shift_'+str(opt_shift_y)+'_'+str(opt_shift_x)+'_orig.nii'))



In [ ]:
def badpixelCorr(img):
    valid_mask = (img != 0.0)
    print(np.sum(valid_mask))
    coords = np.array(np.nonzero(valid_mask)).T
    values = img[valid_mask]
    it = interpolate.NearestNDInterpolator(coords, values)
    img = it(list(np.ndindex(img.shape))).reshape(img.shape)
    return img

In [ ]:
projectionsCorr = totalproj

In [ ]:
if not os.path.exists(badpixelcorrdirname):
    os.makedirs(badpixelcorrdirname)
    
for slice in range(0,projectionsCorr.shape[0]): 
    projection = projectionsCorr[slice,:,:] 
    im = Image.fromarray(projection)
    im.save(os.path.join(badpixelcorrdirname, 'projection{}.tiff'.format(str(slice).zfill(3))))


In [ ]:
angles = np.linspace(0,2*np.pi*((720-1)/720),720,dtype=np.float32)
print(angles.shape)

angles = angles[0:387]
print(angles.shape)

totalangles = angles




In [ ]:
projectionsCorr = totalproj[0:190,:,:]
angles = totalangles[0:190]

# projectionsCorr = totalproj[80:270,:,:]
# angles = totalangles[80:270]

In [ ]:
kernel = Gaussian2DKernel(x_stddev=2)
for slice in range(0,projectionsCorr.shape[0]): 
    projection = projectionsCorr[slice,:,:]
    projection[mask==1] = np.nan
    projection = interpolate_replace_nans(projection, kernel)
    projectionsCorr[slice,:,:] = projection
#     im.save(os.path.join(projcorrdirname, 'projection{}.tiff'.format(str(slice).zfill(3))))

In [ ]:
yshift = 2
# print(yshift)

projectionsCorrsmall = projectionsCorr[:,int(projectionsCorr.shape[1]/2)-1-yshift:int(projectionsCorr.shape[1]/2)-yshift,:]
# projectionsCorrsmall = projectionsCorr[:,0:int(projectionsCorr.shape[1]/2),:]
print(projectionsCorrsmall.shape)
maxvalue = -9999
opt_shift = -99
N = 100
x = np.zeros(N)
y = np.zeros(N)
cnt =0 
for i in range(5,105,1):
# for i in range(0,1,1):
    geosmall.offDetector = np.array((0, (14.4)*0.055)) 
    geosmall.offOrigin = np.array((0.0, 0, 0)) 
    geosmall.rotDetector = np.array((0,0,0.0))
    geosmall.DSD = 192.57
    stageoffset = 0
    geosmall.DSO = geosmall.DSD-43.74 - stageoffset
    
    imgfdk=algs.fdk(projectionsCorrsmall,geosmall,angles)
#     imgfdk = tomopy.misc.corr.remove_ring(imgfdk, center_x=None, center_y=None, thresh=100.0, thresh_max=100.0, thresh_min=- 100.0, theta_min=i, rwidth=16, int_mode='WRAP', ncore=None, nchunk=None, out=None)
    imgfdk = tomopy.misc.corr.remove_ring(imgfdk, center_x=None, center_y=None, thresh=200.0, thresh_max=300.0, thresh_min=40.0, theta_min=50, rwidth=15, int_mode='WRAP', ncore=None, nchunk=None, out=None)
    imgfdk[imgfdk < 0] = 0
    im = imgfdk[0,:,:].astype(np.float32)
#     print(np.mean(im))
#     plt.imshow(im,cmap='gray')
#     plt.show()
    dft = cv2.dft(im, flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
    value = np.mean(magnitude_spectrum)
    if value > maxvalue:
        maxvalue = value
        optshift = i
    print(i, np.mean(magnitude_spectrum))
    x[cnt] = i
    y[cnt] = value
    cnt += 1
print(optshift, maxvalue)
plt.scatter(x, y)
plt.show()

In [ ]:
import SimpleITK as sitk
print(projectionsCorr.shape)
newarr = np.diff(projectionsCorr, axis=0)
newarr = np.diff(newarr, axis=0)

print(newarr.shape)
maxvalues = np.max(np.abs(newarr), axis=0)
sitk.WriteImage(sitk.GetImageFromArray(maxvalues), os.path.join(resultimages, 'what_max.nii'))
print(np.mean(maxvalues), np.std(maxvalues))
maxvalues[maxvalues < (np.mean(maxvalues)+6*np.std(maxvalues))]=0
sitk.WriteImage(sitk.GetImageFromArray(maxvalues), os.path.join(resultimages, 'what_max_th.nii'))
maxvalues[maxvalues> 0] = 1
mask = maxvalues

In [ ]:
yshift = int(6/0.055)
print(yshift)


# projectionsCorrsmall = projectionsCorr[:,0:int(projectionsCorr.shape[1]/2),:]
print(projectionsCorrsmall.shape)
maxvalue = -9999
opt_shift = -99
N = 40
x = np.zeros(N)
y = np.zeros(N)
cnt =0 
# for i in range(-50,50,1):
for i in range(-20,20,1):
    projectionsCorrsmall = projectionsCorr[:,int(projectionsCorr.shape[1]/2)-1-yshift:int(projectionsCorr.shape[1]/2)-yshift,:]
    geosmall.offDetector = np.array((0, i*0.055)) 
    geosmall.offOrigin = np.array((0, 0, 0)) 
    geosmall.rotDetector = np.array((0,0,0.0))
    geosmall.DSD = 192.57
    stageoffset = 0
    geosmall.DSO = geosmall.DSD-43.74 - stageoffset
    
    imgfdk=algs.fdk(projectionsCorrsmall,geosmall,angles)
    imgfdk = tomopy.misc.corr.remove_ring(imgfdk, center_x=None, center_y=None, thresh=100.0, thresh_max=100.0, thresh_min=- 100.0, theta_min=100, rwidth=i, int_mode='WRAP', ncore=None, nchunk=None, out=None)
#     print(i, np.mean(imgfdk))
    im = imgfdk[0,:,:].astype(np.float32)
    print(np.mean(im))
#     plt.imshow(im,cmap='gray')
#     plt.show()
    dft = cv2.dft(im, flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
    value = np.mean(magnitude_spectrum)
    if value > maxvalue:
        maxvalue = value
        optshift = i
    print(i, np.mean(magnitude_spectrum))
    x[cnt] = i
    y[cnt] = value
    cnt += 1
print(optshift, maxvalue)
plt.scatter(x, y)
plt.show()

In [ ]:
import tomopy
projectionsCorr = tomopy.prep.stripe.remove_stripe_fw(totalproj, level=None, wname='db5', sigma=0.2, pad=True, ncore=None, nchunk=None)

In [ ]:
# Stripe removal on the sinogram
import algotom
import algotom.util.utility as utili
import algotom.prep.correction as corr
import algotom.prep.removal as remo
import algotom.prep.filtering as filt
def cudaRun(dataProj):
    dataProj1 = np.empty_like(dataProj);
    for i in range(0, dataProj.shape[1]):
        sinogram = dataProj[:,i,:];
#         sino_360 = remo.remove_all_stripe(sinogram, snr=1.4, la_size=10, sm_size=5);
        # size=17, dim=12 FOR TISSUE BLOCKS 1
#         sino_360 = remo.remove_stripe_based_sorting(sinogram, size=17, dim=12)#,options={"method": "gaussian_filter","para1":(5,9)}) 
        sino_360 = remo.remove_stripe_based_sorting(sinogram, size=19, dim=12)#,options={"method": "gaussian_filter","para1":(5,9)}) 
        # THE ABOVE LINE Works well -- But needs slight optimization
#         sino_360 = remo.remove_stripe_based_filtering(sino_360, sigma=2, size=10, dim=10, sort=True)
#         sino_360 = algotom.prep.removal.remove_stripe_based_fitting(sino_360, order=2, sigma=10, sort=True, num_chunk=1)
#         sino_360 = remo.remove_stripe_based_2d_filtering_sorting(sinogram, sigma=1, size=9, dim=2)
        dataProj1[:,i,:] = sino_360;
        print('Game Over:', i)
    return dataProj1

In [ ]:
projectionsCorr = totalproj

# ni_img = nib.Nifti1Image(projectionsCorr, np.eye(4))
# nib.save(ni_img,  os.path.join(resultimages,'sinogram.nii'))

# dataProj1 = cudaRun(projectionsCorr)
# projectionsCorr = dataProj1
# ni_img = nib.Nifti1Image(projectionsCorr, np.eye(4))
# nib.save(ni_img,  os.path.join(resultimages,'sinogram_clean.nii'))
print(angles.shape)

In [ ]:
maxvalue = -9999
opt_shift = -99
N = 10
x = np.zeros(N)
y = np.zeros(N)
stageoffset = 0
# shiftlist = [-37, -37.5,-38, -38.5,-39, -39.5,-40, -40.5,-41, -41.5] 
# shiftlist = [0] 
# shiftlist = [-39, -38, -37, -36, -35, -34, -33, -32, -31, -30] 
shiftlist = [-6] 
i = 0
for optshift in shiftlist:

    geo.offDetector = np.array((6, optshift*0.055)) 
    geo.offOrigin = np.array((0, 0, 0)) 
#     geo.offOrigin = np.array((-1.0, -0.2, 1.2)) #cc, lr, ap, Block 2 zoomed center
    geo.rotDetector = np.array((0,0,0.0))

    geo.DSD = 192.57
    # geo.DSD = 186 
    stageoffset = 0 
    geo.DSO = geo.DSD-43.74 - stageoffset
    # geo.DSO = geo.DSD-37.2 - stageoffset
#     geo.DSD = 200
#     geo.DSO = 140

    geo.nVoxel = np.array((512, 512, 512))             # number of voxels              (vx)
    geo.sVoxel = np.array((25.6, 25.6, 25.6))          # total size of the image       (mm)
#     geo.sVoxel = np.array((12.8, 12.8, 12.8))          # total size of the image       (mm)
    # geo.sVoxel = np.array((15, 15, 15))          # total size of the image       (mm)
#     geo.sVoxel = np.array((8, 8, 8))          # total size of the image       (mm)
    geo.dVoxel = geo.sVoxel/geo.nVoxel                 # size of each voxel            (mm)


    start = time.time()
    # imgfdk=tigre.algorithms.FDK(projectionsCorr,geo,angles, filter='cosine')
    imgfdk=algs.fdk(projectionsCorr,geo,angles)#,filter='cosine')
    end = time.time()
    print(end - start)
#     cx = geo.sVoxel[0]/2+geo.offOrigin[0]
#     cy = geo.sVoxel[1]/2+geo.offOrigin[1]
#     cy = 3.8
#     cx = 5.2
#     imgfdk = tomopy.misc.corr.remove_ring(imgfdk, center_x=None, center_y=None, thresh=100.0, thresh_max=100.0, thresh_min=- 100.0, theta_min=30, rwidth=20, int_mode='WRAP', ncore=None, nchunk=None, out=None)

    imgfdk[imgfdk < 0] = 0
    
    
#     im = imgfdk[180,:,:].astype(np.float32)
#     dft = cv2.dft(im, flags = cv2.DFT_COMPLEX_OUTPUT)
#     dft_shift = np.fft.fftshift(dft)

#     magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
#     value = np.mean(magnitude_spectrum)
#     if value > maxvalue:
#         maxvalue = value
#         opt_shift = optshift
#     print(i, np.mean(magnitude_spectrum))
#     x[i] = optshift
#     y[i] = value
    
#     im = Image.fromarray(im)
#     imgplot = plt.imshow(im, cmap='gray')
#     im.save(os.path.join(resultimages, 'motion' + str(optshift)+'.tif'))
#     plt.show()
    
    
    
    # imgfdk[imgfdk > 0.1] = 0
    imgfdkint = (imgfdk*3000).astype(np.int32)
    imgfdkint = np.swapaxes(imgfdkint, 1, 2)
    imgfdkint = np.swapaxes(imgfdkint, 0, 2)
    imgfdkint = np.flip(imgfdkint, 2)

    # header = nib.Nifti1Header()
    # header['pixdim'][1:4]  = geo.dVoxel
    # ni_img = nib.Nifti1Image(imgfdkint, np.eye(4), header)
    ni_img = nib.Nifti1Image(imgfdkint, np.eye(4))
    factor = 1.0#/geo.dVoxel[0]
    ni_img.header['pixdim'][1:4]  = geo.dVoxel *factor
    nib.save(ni_img,  os.path.join(resultimages,'recon_shift_'+str(optshift)+'_final.nii'))
    i += 1
# print(opt_shift, maxvalue)
# plt.scatter(x, y)
# plt.show()

In [ ]:
# imgfdk = tomopy.misc.corr.remove_ring(imgfdk, center_x=None, center_y=None, thresh=100.0, thresh_max=100.0, thresh_min=- 100.0, theta_min=30, rwidth=20, int_mode='WRAP', ncore=None, nchunk=None, out=None)

In [ ]:
gradient = np.gradient(projectionsCorr, axis=0, edge_order=2 )
print(gradient.shape)

ni_img = nib.Nifti1Image(gradient, np.eye(4))
nib.save(ni_img,  os.path.join(projdirname,'gradient.nii'))

# mapimg = np.amax(gradient, axis=0) - np.amin(gradient, axis=0)
mapimg = np.std(gradient, axis=0)
im = Image.fromarray(mapimg)
im.save(os.path.join(projdirname, 'gradient.tiff'))


print(np.min(mapimg),np.max(mapimg))

mapimg = mapimg - np.min(mapimg)
mapimg = (mapimg/np.max(mapimg))
im = Image.fromarray(mapimg)
im.save(os.path.join(projdirname, 'gradient2.tiff'))
print(np.mean(mapimg), np.std(mapimg), np.mean(mapimg)+3*np.std(mapimg), np.percentile(mapimg, 99.98))

binary_mask = mapimg > np.percentile(mapimg, 99.98)#(np.mean(mapimg)+5*np.std(mapimg))
im = Image.fromarray(binary_mask)
im.save(os.path.join(projdirname, 'mask.tiff'))
mapimg[binary_mask] = np.nan;
# selection = np.zeros_like(mapimg)
# selection[binary_mask] = mapimg[binary_mask]
im = Image.fromarray(mapimg)
im.save(os.path.join(projdirname, 'gradient3.tiff'))

In [ ]:
os.makedirs(projcorrdirname)
for slice in range(0,projectionsCorr.shape[0]): 
    projection = projectionsCorr[slice,:,:]
    im = Image.fromarray(projection)
    im.save(os.path.join(projcorrdirname, 'projection{}.tiff'.format(str(slice).zfill(3))))

In [ ]:
for slice in range(0,openimages.shape[0]): 
    projection = openimages[slice,:,:]
    im = Image.fromarray(projection)
    im.save(os.path.join(openfielddirname, 'openfield{}.tiff'.format(str(slice).zfill(3))))

In [ ]:
projectionsMatch = np.empty_like(projectionsCorr, dtype=np.float64)
for slice in range(0,projectionsCorr.shape[0]): 
    sx = ndimage.sobel(projectionsCorr[slice,:,:], axis=0, mode='constant')
    sy = ndimage.sobel(projectionsCorr[slice,:,:], axis=1, mode='constant')
    projection = np.hypot(sx, sy)
    projectionsMatch[slice,:,:] = projection
    
motionfield = np.sum(projectionsMatch, axis = 1)
# motionfield1[motionfield1 <50]=0
motionfield = ndimage.gaussian_filter(motionfield, sigma=10)
im = Image.fromarray(motionfield)
im.save(os.path.join(resultimages, 'motion.tif'))
imgplot = plt.imshow(im, cmap='gray')

In [ ]:

result = np.zeros(shape=(motionfield.shape[0]-1))
ref = np.reshape(motionfield[150,:], (1,motionfield.shape[1]))
geooff = np.array((0,0))
count = 0
for slice in range(1,motionfield.shape[0]): 
    count += 1
    projection = np.reshape(motionfield[slice,:], (1,motionfield.shape[1]))
    shift, error, diffphase = phase_cross_correlation(ref, projection, upsample_factor=1000)
    result[count-1] = shift.item(1)


In [ ]:
tt = np.linspace(0, result.shape[0], result.shape[0])
res = fit_sin(tt, result)
print( "Amplitude=%(amp)s, Angular freq.=%(omega)s, phase=%(phase)s, offset=%(offset)s, Max. Cov.=%(maxcov)s" % res )
# data_fit = est_amp*np.sin(est_freq*t+est_phase) + est_mean
pylab.plot(tt, result, '.')
pylab.plot(tt, res["fitfunc"](tt), "r-", label="y fit curve", linewidth=2)
pylab.legend()
pylab.show()

In [ ]:
projectionsCorr2 = np.empty_like(projectionsCorr, dtype=np.float64)

print(projectionsCorr.shape)
for slice in range(0,projectionsCorr.shape[0]):
    img = projectionsCorr[slice,:,:]

    img[binary_mask] = 0;
    im = Image.fromarray(img)
    im.save(os.path.join(projdirname,'test', 'projection{}.tiff'.format(str(slice).zfill(3))))
    img = badpixelCorr(img)
    projectionsCorr2[slice,:,:] = img

In [ ]:
projection0 = projectionsCorr[0,:,:]
projection180 = np.flip(projectionsCorr[round(projectionsCorr.shape[0]/2),:,:],1)

shift, error, diffphase = phase_cross_correlation(projection0, projection180)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(projection0, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')
ax2.imshow(projection180.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Show the output of a cross-correlation to show what the algorithm is
# doing behind the scenes
image_product = np.fft.fft2(projection0) * np.fft.fft2(projection180).conj()
cc_image = np.fft.fftshift(np.fft.ifft2(image_product))
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Cross-correlation")

plt.show()

print(f"Detected pixel offset (y, x): {shift}")
print(error)

# subpixel precision
shift, error, diffphase = phase_cross_correlation(projection0, projection180,
                                                  upsample_factor=100)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(projection0, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(projection180.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Calculate the upsampled DFT, again to show what the algorithm is doing
# behind the scenes.  Constants correspond to calculated values in routine.
# See source code for details.
cc_image = np.fft.fftshift(np.fft.ifft2(image_product))
# cc_image = _upsampled_dft(image_product, 150, 100, (shift*100)+75).conj()
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Supersampled XC sub-area")


plt.show()

print(f"Detected subpixel offset (y, x): {shift}")
print(error)

In [ ]:
# geo.offDetector = np.array((0, -(10)*0.055)) 
geo.offDetector = np.array((0, -(shift.item(1)/2)*0.055)) 

geo.rotDetector = np.array((0,0,0.0))
stageoffset = 0
geo.DSD = 186
geo.DSO = geo.DSD-37.2 - stageoffset


# geo.nVoxel = np.array((1024, 512, 512))             # number of voxels              (vx)
# geo.sVoxel = np.array((25.6, 12.8, 12.8))          # total size of the image       (mm)
# geo.dVoxel = geo.sVoxel/geo.nVoxel                 # size of each voxel            (mm)

# geo.nVoxel = np.array((512, 512, 512))             # number of voxels              (vx)
# geo.sVoxel = np.array((25.6, 25.6, 25.6))          # total size of the image       (mm)
# geo.dVoxel = geo.sVoxel/geo.nVoxel                 # size of each voxel            (mm)

geo.offOrigin = np.array((0,0,0))                 # Offset of image from origin   (mm)
# geo.DSO = 70
angles =np.linspace(0,-2*np.pi*((projectionsCorr.shape[0]-1)/projectionsCorr.shape[0]),projectionsCorr.shape[0],dtype=np.float32)
start = time.time()
imgfdk=tigre.algorithms.FDK(projectionsCorr,geo,angles)
end = time.time()
print(end - start)
tigre.plotimg(imgfdk,slice=round(imgfdk.shape[1]/2),dim='y')
imgfdk[imgfdk < 0] = 0
# imgfdk[imgfdk > 0.1] = 0
imgfdkint = (imgfdk*10000)
# imgfdkint = (imgfdk*1000).astype(np.int32)
header = nib.Nifti1Header()
header['pixdim'][1:4]  = geo.dVoxel
ni_img = nib.Nifti1Image(imgfdkint, np.eye(4), header)
ni_img.header['pixdim'][1:4]  = geo.dVoxel
nib.save(ni_img,  os.path.join(resultimages,'reconfinaltest.nii'))

In [ ]:
tigre.plot_geometry(geo, 0)

In [ ]:
from skimage.measure import regionprops


# for slice in [220, 310, 420]:
for slice in range(200,800):#[220, 310, 420]:

    circleimg = imgfdkint[slice,:,:]
    im = Image.fromarray(circleimg)
    imgplot = plt.imshow(circleimg, cmap='gray')
    
    blobs = circleimg > (np.percentile(circleimg, 98)/2.5)

    # label connected regions that satisfy this condition
    labels, nlabels = ndimage.label(blobs)

#     properties = regionprops(labels)
#     for p in properties:
#         min_row, min_col, max_row, max_col = p.bbox
#         print(max(max_row - min_row, max_col - min_col)*geo.dVoxel[0])

    dt = ndimage.distance_transform_edt(blobs)
    slices = ndimage.find_objects(input=labels)
    radii = [np.amax(dt[s]) for s in slices]

    print(radii[0]*2*geo.dVoxel[0])


In [ ]:
print(dt)

In [ ]:
print(header)

drive = 'C:\\'
fixedpath = os.path.join(drive,'Data', 'fixed.nii')
fixed = nib.load(fixedpath)

In [ ]:
print(fixed.affine)
print(ni_img.affine)

In [ ]:
imgfdk[imgfdk < 0] = 0
# imgfdk[imgfdk > 0.1] = 0
print(imgfdk.dtype)
imgfdkint = (imgfdk*1000).astype(np.int32)
print(imgfdkint.dtype)
imgfdksmall = imgfdkint[:,:,:]
ni_img = nib.Nifti1Image(imgfdksmall, np.eye(4))
dirname = 'C:\\'
dirname = os.path.join(dirname,'data', 'Align2')
nib.save(ni_img,  os.path.join(dirname,'test6.nii'))

In [ ]:
angles =np.linspace(0,2*np.pi*((projections.shape[0]-1)/projections.shape[0]),projections.shape[0],dtype=np.float32)
dirname = 'C:\\'
dirname = os.path.join(dirname,'Dat', '360')
# geosmall.offdetector = np.array((0, (-6.68/2)*0.0495)) 
geosmall.offdetector = np.array((0, 0)) 
# distlist1 = [170,185,190,195,200,205]
distlist1 = [170,175]
distlist2 = [60,65,70]
for dist1 in distlist1:
    for dist2 in distlist2:
        geosmall.DSD = dist1
        geosmall.DSO = dist1-dist2
        start = time.time()
    # imgfdk=tigre.algorithms.FDK(projections,geo,angles)
        imgfdk=tigre.algorithms.FDK(projections,geosmall,angles)
        end = time.time()
#         print(dist, ':' , end - start)
        imgfdk[imgfdk < 0] = 0
        imgfdk = (imgfdk*1000).astype(np.int32)
        sumscore = 0
        count = 0
        for i in range(150,200):
            gy, gx = np.gradient(imgfdk[i,:,:])
            gnorm = np.sqrt(gx**2 + gy**2)
            sumscore = sumscore + np.average(gnorm)
            count += 1
        print(dist1, dist2, sumscore/count, sep='\t')
        ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
        nib.save(ni_img,  os.path.join(dirname,'test{}_{}.nii'.format(str(dist1), str(dist2))))
# tigre.plotimg(imgfdk,slice=128,dim='y')

In [ ]:
tigre.plotimg(imgfdk,slice=200,dim='z')

slicenr = imgfdk[200,:,:]
#     projection = ndimage.median_filter(projection, 5)
#     sx = ndimage.sobel(projection, axis=0, mode='constant')
#     sy = ndimage.sobel(projection, axis=1, mode='constant')
#     projection = np.hypot(sx, sy)
#     projection = np.gradient(projections[slice,:,:], axis=1)
im = Image.fromarray(slicenr)
im.save(os.path.join(dirname, 'slice{}.tiff'.format(str(200).zfill(3))))

In [ ]:
projectionssmall = projectionsCorr[:,0:500,:]
projections = projectionsCorr
# geo.offdetector = geooff
# geosmall.offdetector = geooff

In [ ]:
print(geooff.shape)
geooff = np.reshape(geooff, (500, 2))
# print(geooff)
print(np.average(geooff, axis=0))
geooff = np.average(geooff, axis=0)
print(geooff)

In [ ]:
sumscore = 0
count = 0
for i in range(150,200):
    gy, gx = np.gradient(imgfdk[i,:,:])
    gnorm = np.sqrt(gx**2 + gy**2)
    sumscore = sumscore + np.average(gnorm)
    print(np.average(gnorm))
    count += 1
print(sumscore/count)

In [ ]:
print(dirname)

In [ ]:
from scipy import ndimage
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
dirname = os.path.join(dirname,'Align')
dirname = os.path.join(dirname,'projectionimg')
for slice in range(0,projections.shape[0]): 
    projection = projectionsCorr[slice,:,:]
#     projection = ndimage.median_filter(projection, 5)
#     sx = ndimage.sobel(projection, axis=0, mode='constant')
#     sy = ndimage.sobel(projection, axis=1, mode='constant')
#     projection = np.hypot(sx, sy)
#     projection = np.gradient(projections[slice,:,:], axis=1)
    im = Image.fromarray(projection)
    im.save(os.path.join(dirname, 'projection{}.tiff'.format(str(slice).zfill(3))))

In [ ]:
# projectionref = np.gradient(projections[0,:,:], axis=1)*1000
projectionref = projections[0,:,:]
for slice in range(1,projections.shape[0]): 
    if (slice % 1 == 0):
#         projection = np.gradient(projections[slice,:,:], axis=1)*1000
        projection = projections[slice,:,:]
        shift, error, diffphase = phase_cross_correlation(projectionref, projection,
                                                  upsample_factor=1000)
        print(slice, '\t', shift.item(1))
#         projectionref = projection

In [ ]:
dirname = 'C:\\'
dirname = os.path.join(dirname,'Dat')
dirname = os.path.join(dirname,'360')
dirname = os.path.join(dirname,'projectionimg')
if not os.path.exists(dirname):
    os.makedirs(dirname)
for slice in range(0,projections.shape[0]): 
    projection = projections[slice,:,:] 
    im = Image.fromarray(projection)
    im.save(os.path.join(dirname, 'projection{}.tif'.format(str(slice).zfill(3))))
print(projections.shape)
print(openimages.shape)

In [ ]:
tigre.plotimg(imgfdkint,slice=256,dim='y')

In [ ]:
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
dirname = os.path.join(dirname,'projections')
for slice in range(0,projections.shape[0]): 
    projection = projections[slice,:,:] 
    im = Image.fromarray(projection)
    im.save(os.path.join(dirname, 'projection{}.tif'.format(str(slice).zfill(3))))

In [ ]:
projectionsmall = projections[:,np.int(projections.shape[1]/2)-1:np.int(projections.shape[1]/2), :]

imgfdksmall=tigre.algorithms.FDK(projectionsmall,geosmall,angles)
tigre.plotimg(imgfdksmall,slice=0,dim='z')
# for offset in range(-20,20): 
# # for offset in range(0,10): 
#     corr = offset/10
#     geosmall.offOrigin = np.array((0,0,0))
#     geosmall.offDetector = np.array((0, 0))                 # Offset of Detector            (mm) 
#     imgfdksmall=tigre.algorithms.FDK(projectionsmall,geosmall,angles)
#     tigre.plotimg(imgfdksmall,slice=0,dim='z')
#     print(corr)


In [ ]:
for offset in range(20,30): 
# for offset in range(0,10): 
    corr = (offset/1000)
    projectionstest = projectionsmall - corr
    imgfdksmall=tigre.algorithms.FDK(projectionstest,geosmall,angles)
    tigre.plotimg(imgfdksmall,slice=0,dim='z')
    print(corr)

In [ ]:
np.percentile(projectionsmall, 1)


In [ ]:
import time
start = time.time()
geo.offDetector = np.array((0, 1.782))                 # Offset of Detector            (mm)
imgfdk=tigre.algorithms.FDK(projections,geo,angles)
end = time.time()
print(end - start)
tigre.plotimg(imgfdk,slice=256,dim='y')

In [ ]:
tigre.plotimg(imgfdk,slice=345,dim='y')

In [ ]:

imgfdk = imgfdk*1000
ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
nib.save(ni_img,  os.path.join(dirname,'reconFDKIntauto.nii'))

In [ ]:
print(np.percentile(imgfdk, 5), np.min(imgfdk), np.max(imgfdk), np.percentile(imgfdk, 99.5))

In [ ]:
imgfdk[imgfdk < 0] = 0
# imgfdk[imgfdk > 0.1] = 0
print(imgfdk.dtype)
imgfdkint = (imgfdk*10000).astype(np.int16)
print(imgfdkint.dtype)
ni_img = nib.Nifti1Image(imgfdkint, np.eye(4))
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
nib.save(ni_img,  os.path.join(dirname,'reconFDKintHR.nii'))

In [ ]:
tigre.plotimg(imgfdksmall,slice=0,dim='z')
self.offDetector = np.array((0, -2.5))                 # Offset of Detector            (mm)

In [ ]:
imgfdk=tigre.algorithms.FDK(projections,geo,angles)
tigre.plotimg(imgfdk,slice=126,dim='x')

In [ ]:
import itk

In [ ]:
from skimage.transform import warp_polar, rotate, rescale
radius=128
image_polar = warp_polar(projection0, radius=radius, multichannel=False)
rotated_polar = warp_polar(projection180, radius=radius, multichannel=False)
shifts, error, phasediff = phase_cross_correlation(image_polar, rotated_polar)
# print("Expected value for counterclockwise rotation in degrees: "
#       f"{angle}")
print("Recovered value for counterclockwise rotation: "
      f"{shifts[0]}")

In [ ]:
print(np.int(projections.shape[1]/2))
projectionsmall = projections[:,np.int(projections.shape[1]/2)-1:np.int(projections.shape[1]/2)+1, :]
slice35 = projections[35,:,:]
print(projections.shape)
print(projectionsmall.shape)
# print(np.max(projections))
# print(np.min(projections))
# projectionsmall = projections[:,int]


In [ ]:
# imgfdk[imgfdk < 0] = 0

#     ni_img = nib.Nifti1Image(imgfdk, np.eye(4))

In [ ]:
maxsharp = 0
best =0
for offset in range(200,300): 
# for offset in range(0,10): 
    corr = -(offset/100)
    geosmall.offDetector = np.array((0, corr))                 # Offset of Detector            (mm) 
    imgfdk=tigre.algorithms.FDK(projectionsmall,geosmall,angles)
#     imgfdk=tigre.algorithms.FDK(projections,geo,angles)
    imgfdk[imgfdk < 0] = 0
    gy, gx = np.gradient(imgfdk[0,:,:])
    gnorm = np.sqrt(gx**2 + gy**2)
    sharpness1 = np.average(gnorm)
    gy, gx = np.gradient(imgfdk[1,:,:])
    gnorm = np.sqrt(gx**2 + gy**2)
    sharpness2 = np.average(gnorm)
    sharpness = (sharpness1 + sharpness2)/2
    
    if sharpness>maxsharp:
        maxsharp = sharpness
        best = corr/1000
    print(corr, sharpness, sep='\t')
#     tigre.plotimg(imgfdk,slice=0,dim='z')
    ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
    nib.save(ni_img,  os.path.join(os.getcwd(),'data', 'reconslice{}.nii'.format(str(offset).zfill(3))))
print(best, maxsharp, sep='\t')

In [ ]:
print(imgfdk.shape)
gy, gx = np.gradient(imgfdk[0,:,:])
gnorm = np.sqrt(gx**2 + gy**2)
sharpness = np.average(gnorm)
print(sharpness)

In [ ]:
# geo.rotDetector = np.array((0,0,0))
imgfdk=tigre.algorithms.FDK(projections,geo,angles)
imgfdk = np.rint(imgfdk*1000)
imgfdk[imgfdk < 0] = 0

# Show the results
tigre.plotimg(imgfdk,slice=124,dim='x')
ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
nib.save(ni_img, 'outputoffsetnavrit.nii')

geo.offDetector = np.array((0, -2.4684))                 # Offset of Detector            (mm)
imgfdk=tigre.algorithms.FDK(projections,geo,angles)
imgfdk = np.rint(imgfdk*1000)
imgfdk[imgfdk < 0] = 0
# Show the results
tigre.plotimg(imgfdk,slice=124,dim='x')
ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
nib.save(ni_img, 'outputoffsetjasper.nii')

# from tigre.utilities import im_3d_denoise

# imgfdk=tigre.algorithms.FDK(projections,geo,angles)
# # imgfdk=tigre.algorithms.FDK(projections,geo,angles,filter='shepp_logan')
# # Show the results
# tigre.plotimg(imgfdk,slice=220,dim='z')
# imgfdk = np.rint(imgfdk*1000)
# imgfdk[imgfdk < 0] = 0
# ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
# nib.save(ni_img, 'output_FDK_1.nii')

# imgfdk2=tigre.algorithms.FDK(projections,geo,angles)
# # Show the results
# tigre.plotimg(imgfdk2,slice=128,dim='x')

# ni_img = nib.Nifti1Image(imgfdk2, np.eye(4))
# nib.save(ni_img, 'output.nii.gz')

In [ ]:
print(os.path.join(os.getcwd(),'data', 'recon{}.nii'.format(str(offset).zfill(3))))
os.getcwdb()

In [ ]:
projections32 = projections.astype(np.float32)
imgSARTTV = tigre.algorithms.ossart_tv(projections32,geo,angles,niter=100,tviter=50,tvlambda=5000,blocksize=100)
imgSARTTV = np.rint(imgSARTTV*1000)
imgSARTTV[imgSARTTV < 0] = 0
ni_img = nib.Nifti1Image(imgSARTTV, np.eye(4))
nib.save(ni_img, 'output_imgSARTTV.nii')

In [ ]:
# geo.offDetector = np.array((0, -2.53))                 # Offset of Detector            (mm)

niter=50
imgOSSART=tigre.algorithms.ossart(projections,geo,angles,niter, **dict(blocksize=100))
imgOSSART = np.rint(imgOSSART*1000)
imgOSSART[imgOSSART < 0] = 0
ni_img = nib.Nifti1Image(imgOSSART, np.eye(4))
nib.save(ni_img, 'output_imgOSSART2.nii')

In [ ]:
niter=30
imgSART=tigre.algorithms.sart(projections,geo,angles,niter)
tigre.plotimg(imgSART,slice=124,dim='x')

ni_img = nib.Nifti1Image(imgSART, np.eye(4))
nib.save(ni_img, 'outputSart30.nii.gz')


# niter=15
# imgSART=tigre.algorithms.sart(projections,geo,angles,niter, **dict(init="FDK"))
# tigre.plotimg(imgSART,slice=124,dim='x')


In [ ]:
print(np.min(projections), np.max(projections))
niter=15
imgCGLS=tigre.algorithms.cgls(projections,geo,angles,niter, **dict(alpha=125))

In [ ]:
tigre.plotimg(imgOSSART,slice=128,dim='x')
ni_img = nib.Nifti1Image(imgOSSART, np.eye(4))
nib.save(ni_img, 'reconOSSART.nii.gz')

In [ ]:
from PIL import Image
basename='ffCorr_th0_4-7keV_'
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
dirname = os.path.join(dirname,'CT_recon_500-1')
print(dirname)

final = []
for fname in os.listdir(dirname):
    im = Image.open(os.path.join(dirname, fname))
    imarray = np.array(im)
#         dead_pixels = np.isinf(imarray)
#         blurred = median_filter(imarray, size=2, mode="mirror")
#         filtered = np.ma.where(dead_pixels, blurred, imarray)
    final.append(imarray)

final = np.asarray(final) # shape = (60000,28,28)
print(final.shape)

ni_img = nib.Nifti1Image(final, np.eye(4))


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import median_filter
for number in range(34,35): # Run through all patients
    filename = os.path.join(dirname, '{}{}.tif'.format(basename, str(number).zfill(3)))
    im = Image.open(filename)
    plt.plot(im)
    pix = np.array(im)
    dead_pixels = np.isinf(pix)
    print(dead_pixels)
    print(number, np.min(pix), np.max(pix), np.mean(pix))
    blurred = median_filter(pix, size=2, mode="mirror")
    filtered = np.ma.where(dead_pixels, blurred, pix)
    print(number, np.min(filtered), np.max(filtered), np.mean(filtered)) 
    img = Image.fromarray(pix)
    print(pix.dtype)
    imgplot = plt.imshow(im)
#     plt.plot(img)
#     plt.show()

# print(filename)
# from PIL import Image
# # import matplotlib.pyplot as plt
# # import tifffile

# # im1=tifffile

# im = Image.open(filename)
# # print(im.size)

# pix = np.array(im)
# print(np.max(pix))
# print(np.min(pix))
# # print(pix.dtype)
# # print(pix.shape)
# # plt.plot(pix)
# # plt.show()

In [ ]:
# import tifffile
# img = tifffile.imread(filename)
# print(img.shape)
# plt.plot(img)
# plt.show()


In [ ]:
tigre.plotimg(projections,slice=35,dim='z')
slice35 = projections[35,:,:]
print(slice35.shape)
print(np.min(slice35), np.mean(slice35), np.max(slice35))

In [ ]:
from PIL import Image
im = Image.fromarray(slice35)
im.save('slice35_navrit.tif')
from scipy.ndimage import median_filter
blurred = median_filter(slice35, size=2)
im.save('slice35_navrit2.tif')
print(np.min(blurred), np.mean(blurred), np.max(blurred))
im2 = abs(slice35 - blurred)

In [ ]:
print(np.min(im2), np.mean(im2), np.max(im2))

In [ ]:
def find_outlier_pixels(data,tolerance=3,worry_about_edges=True):
    #This function finds the hot or dead pixels in a 2D dataset. 
    #tolerance is the number of standard deviations used to cutoff the hot pixels
    #If you want to ignore the edges and greatly speed up the code, then set
    #worry_about_edges to False.
    #
    #The function returns a list of hot pixels and also an image with with hot pixels removed

    from scipy.ndimage import median_filter
    blurred = median_filter(data, size=2)
    difference = data - blurred
    threshold = 10*np.std(difference)

    #find the hot pixels, but ignore the edges
    hot_pixels = np.nonzero((np.abs(difference[1:-1,1:-1])>threshold) )
    hot_pixels = np.array(hot_pixels) + 1 #because we ignored the first row and first column

    fixed_image = np.copy(data) #This is the image with the hot pixels removed
    for y,x in zip(hot_pixels[0],hot_pixels[1]):
        fixed_image[y,x]=blurred[y,x]

    if worry_about_edges == True:
        height,width = np.shape(data)

        ###Now get the pixels on the edges (but not the corners)###

        #left and right sides
        for index in range(1,height-1):
            #left side:
            med  = np.median(data[index-1:index+2,0:2])
            diff = np.abs(data[index,0] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[index],[0]]  ))
                fixed_image[index,0] = med

            #right side:
            med  = np.median(data[index-1:index+2,-2:])
            diff = np.abs(data[index,-1] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[index],[width-1]]  ))
                fixed_image[index,-1] = med

        #Then the top and bottom
        for index in range(1,width-1):
            #bottom:
            med  = np.median(data[0:2,index-1:index+2])
            diff = np.abs(data[0,index] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[0],[index]]  ))
                fixed_image[0,index] = med

            #top:
            med  = np.median(data[-2:,index-1:index+2])
            diff = np.abs(data[-1,index] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[height-1],[index]]  ))
                fixed_image[-1,index] = med

        ###Then the corners###

        #bottom left
        med  = np.median(data[0:2,0:2])
        diff = np.abs(data[0,0] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[0],[0]]  ))
            fixed_image[0,0] = med

        #bottom right
        med  = np.median(data[0:2,-2:])
        diff = np.abs(data[0,-1] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[0],[width-1]]  ))
            fixed_image[0,-1] = med

        #top left
        med  = np.median(data[-2:,0:2])
        diff = np.abs(data[-1,0] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[height-1],[0]]  ))
            fixed_image[-1,0] = med

        #top right
        med  = np.median(data[-2:,-2:])
        diff = np.abs(data[-1,-1] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[height-1],[width-1]]  ))
            fixed_image[-1,-1] = med

    return hot_pixels,fixed_image


In [ ]:
hot, fixed35 = find_outlier_pixels(slice35,3,True)

In [ ]:
print(final.shape)

In [ ]:
from PIL import Image
import numpy as np
import os

In [ ]:
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
dirname = os.path.join(dirname,'Autoscan')
filepath = os.path.join(dirname,'Frame_0000_avg.raw')
print(filepath)
rawData = open(filepath, 'rb').read()
imgSize = (1176,1104)
# Use the PIL raw decoder to read the data.
# the 'F;16' informs the raw decoder that we are reading 
# a little endian, unsigned integer 16 bit data.
img = Image.frombytes('I;16L', imgSize, rawData, 'raw')
imarray = np.array(img)
imarray = np.flipud(imarray)
imarray = np.fliplr(imarray)
im = Image.fromarray(imarray)
im.save(os.path.join(dirname, 'self.tif'))

# im = Image.open(filepath)
# with open('filename', 'rb') as f:
#     im = Image.fromstring('L;16', (width, height), f.read()) # also try 'L;16B', 'I;16', and 'I;16B'

In [ ]:
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
dirname = os.path.join(dirname,'test')
# dirname = os.path.join(dirname,'tif')
print(dirname)
projections = data_loader.load_raw_files(dirname)

In [ ]:
projection0 = projections[1,:,:]
projection180 = np.flip(projections[3,:,:],1)
shift, error, diffphase = phase_cross_correlation(projection0, projection180,
                                                  upsample_factor=100)
print(shift.item(1))
offset = -(shift.item(1)/2)*0.0495
print(offset)

In [ ]:
print(projections.shape)
average = projections[51,:,:]
projections = projections[0:51,:,:]
print(projections.shape)
proj1 = projections[0,:,:]


In [ ]:
median = np.median(projections, axis=0)
print(median.shape)

In [ ]:
dirname = 'C:\\'
dirname = os.path.join(dirname,'Data')
im = Image.fromarray(average)
im.save(os.path.join(dirname, 'average.tif'))
im = Image.fromarray(median)
im.save(os.path.join(dirname, 'median.tif'))
im = Image.fromarray(proj1)
im.save(os.path.join(dirname, 'proj1.tif'))



In [ ]:
projections10 = projections[0:10,:,:]
projections20 = projections[0:20,:,:]
projections30 = projections[0:30,:,:]
projections40 = projections[0:40,:,:]
median10 = np.median(projections10, axis=0)
median20 = np.median(projections20, axis=0)
median30 = np.median(projections30, axis=0)
median40 = np.median(projections40, axis=0)
avg10 = np.mean(projections10, axis=0)
avg20 = np.mean(projections20, axis=0)
avg30 = np.mean(projections30, axis=0)
avg40 = np.mean(projections40, axis=0)

print(median.shape)

In [ ]:
im = Image.fromarray(median10)
im.save(os.path.join(dirname, 'median10.tif'))
im = Image.fromarray(median20)
im.save(os.path.join(dirname, 'median20.tif'))
im = Image.fromarray(median30)
im.save(os.path.join(dirname, 'median30.tif'))
im = Image.fromarray(median40)
im.save(os.path.join(dirname, 'median40.tif'))
im = Image.fromarray(avg10)
im.save(os.path.join(dirname, 'avg10.tif'))
im = Image.fromarray(avg20)
im.save(os.path.join(dirname, 'avg20.tif'))
im = Image.fromarray(avg30)
im.save(os.path.join(dirname, 'avg30.tif'))
im = Image.fromarray(avg40)
im.save(os.path.join(dirname, 'avg40.tif'))


In [ ]:
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage import io
 

drive = 'C:\\'
projdirname = os.path.join(drive,'data','Test', 'BPC_images', 'projection000.tif')
if os.path.isfile(projdirname):
    
    img = io.imread(projdirname)
    img2 = ndimage.gaussian_filter(img, 2)
    img3 = ndimage.median_filter(img, 3)
#     img2 = Image.despeckle(img)

# #     sigma_est = estimate_sigma(img, multichannel=False, average_sigmas=True)
# #     print(sigma_est)
# #     img2 = denoise_tv_chambolle(img, weight=0.2, multichannel=True)
# # #     img3 = denoise_bilateral(img, sigma_color=0.05, sigma_spatial=15, multichannel=False)
# #     img4 = denoise_wavelet(img, multichannel=True, rescale_sigma=True)
# # #     img = cv2.imread(projdirname, -1)
    plt.subplot(121),plt.imshow(img,  cmap='gray')
    plt.subplot(122),plt.imshow(img2,   cmap='gray')
    plt.show()
# #     dst= cv2.fastNlMeansDenoising(img,None,10,7,21)
    im = Image.fromarray(img2)
    im.save(os.path.join(drive,'data','Test', 'BPC_images', 'projection000_gauss.tif'))
    im = Image.fromarray(img3)
    im.save(os.path.join(drive,'data','Test', 'BPC_images', 'projection000_median.tif'))
# #     im = Image.fromarray(img4)
# #     im.save(os.path.join(drive,'data','Test', 'BPC_images', 'projection000_wavelet.tif'))
# #     plt.subplot(121),plt.imshow(img)
# #     plt.subplot(122),plt.imshow(dst)
# #     plt.show()

In [ ]:
filepath = projdirname
drive = 'e:\\'
filepath = os.path.join(drive,'data','tumor2', '25', )
outpath = os.path.join(drive,'data','tumor2', 'median', '25')
if not os.path.exists(outpath):
    os.makedirs(outpath)

totalset = np.empty((5, 512, 512), dtype=np.float64)
curimagenr = ''
position = 0
if os.path.exists(filepath):
    final = []
    cnt = 0
    for fname in os.listdir(filepath):
        filename, file_extension = os.path.splitext(fname)

        if file_extension.lower() == '.tiff':
            im = Image.open(os.path.join(filepath, fname))                
#                 print(os.path.join(filepath, fname))
        else:
            continue

        imagenr = filename[0:4]
        
        print(imagenr, curimagenr)
        if curimagenr == '':
            curimagenr = imagenr
        
        
        if curimagenr != imagenr:
            medianimg = np.median(totalset, axis=0)
            im = Image.fromarray(medianimg)
            im.save(os.path.join(outpath, curimagenr + '_25.tiff'))
            position = 0
            
        if curimagenr == imagenr:
            imarray = np.array(im)
            totalset[position,:,:] = imarray
            position = position + 1

        curimagenr = imagenr    
        
            

In [ ]:
drive = 'c:\\'


lst = ['25', '30', '35', '40', '45', '50', '55', '60']
for energy in lst:

    filepath = os.path.join(drive,'data','tumor2', energy)
    outpath = os.path.join(drive,'data','tumor2', 'sum', energy)
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    totalset = np.empty((5, 512, 512), dtype=np.float64)
    curimagenr = ''
    position = 0
    if os.path.exists(filepath):
        final = []
        cnt = 0
        for fname in os.listdir(filepath):
            filename, file_extension = os.path.splitext(fname)

            if file_extension.lower() == '.tiff':
                im = Image.open(os.path.join(filepath, fname))                
    #                 print(os.path.join(filepath, fname))
            else:
                continue

            imagenr = filename[0:4]

            if curimagenr == '':
                curimagenr = imagenr
            
            imarray = np.array(im)
            totalset[position,:,:] = imarray
            position = position + 1
            
            if (position == totalset.shape[0]-1):
                medianimg = np.sum(totalset, axis=0)
                im = Image.fromarray(medianimg)
                im.save(os.path.join(outpath, curimagenr + '_'+ energy + '.tiff'))
                print(os.path.join(outpath, curimagenr + '_'+ energy + '.tiff'))
                position = 0
                curimagenr = ''
                
    filepath = os.path.join(drive,'data','tumor2', energy, 'Openimg')
    outpath = os.path.join(drive,'data','tumor2', 'sum', energy,  'Openimg')
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    totalset = np.empty((5, 512, 512), dtype=np.float64)
    curimagenr = ''
    position = 0
    if os.path.exists(filepath):
        final = []
        cnt = 0
        for fname in os.listdir(filepath):
            filename, file_extension = os.path.splitext(fname)

            if file_extension.lower() == '.tiff':
                im = Image.open(os.path.join(filepath, fname))                
    #                 print(os.path.join(filepath, fname))
            else:
                continue

            imagenr = filename[0:7]

            if curimagenr == '':
                curimagenr = imagenr
            
            imarray = np.array(im)
            totalset[position,:,:] = imarray
            position = position + 1
            
            if (position == totalset.shape[0]-1):
                medianimg = np.sum(totalset, axis=0)
                im = Image.fromarray(medianimg)
                im.save(os.path.join(outpath, curimagenr + '_'+ energy + '.tiff'))
                print(os.path.join(outpath, curimagenr + '_'+ energy + '.tiff'))
                position = 0
                curimagenr = ''
                




In [ ]:
from skimage.filters.rank import median
from skimage.morphology import disk
import scipy.ndimage as ndi
drive = 'c:\\'
filepath = os.path.join(drive,'data','HU calibration', '0500_46.tiff')
# filename = 'C:\Data\HU calibration\HU\HU\46\sum\0000_46.tiff' 
im = np.array(Image.open(filepath))

half = np.int(im.shape[0]/2)
im[half-1] = 0
im[half] = 0
im[:,half-1] = 0
im[:,half] = 0
img_save = Image.fromarray(im)
img_save.save(os.path.join(drive,'data','HU calibration', 'test0.tiff'))
mask = (im > 0)

img_masked = np.where(mask, im, np.nan)

size = 5
img_masked_median = ndi.generic_filter(img_masked, np.nanmedian, size=size)
img_median = Image.fromarray(img_masked_median)
img_median.save(os.path.join(drive,'data','HU calibration', 'test.tiff'))
# img_unmasked_median = ndi.median_filter(img, size=size)
# print(im.dtype)
# med = median(im, disk(5))
# print(med.dtype)